In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label
import math

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [2]:
num_qubits = 2
max_weight = 2
term_dict = {("H", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    union = used_indices_1.union(used_indices_2)
    if len(union) > 0 and len(union) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
sign_iterator = list(product([1,-1], repeat=num_qubits))
prep_string_iterator = product([math.prod([item for item in sign_tuple]) for sign_tuple in sign_iterator],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))
awk_iterator = list(i[0] for i in product(sign_iterator, [p for p in measure_string_iterator]))

In [4]:
list(sign_iterator)

[(1, 1), (1, -1), (-1, 1), (-1, -1)]

In [ ]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)

In [6]:
hs_experiment

[('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, -1), stim.PauliString("+XY")),
  stim.PauliString("+XX")),
 ('h',
  

In [ ]:
import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-5:-1])
            sign_string = el[-4]
            egen = ",".join(str(s) for s in el[:-5])
            coef = int(el[-1])
            # ic(observable)
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

ic| observable: '+XX,(1, 1),+XX,+X_'
ic| observable: '+XX,(1, 1),+XX,+_X'
ic| observable: '+XX,(1, 1),+XX,+XX'
ic| observable: '+XX,(1, 1),+XY,+X_'
ic| observable: '+XX,(1, 1),+XY,+_Y'
ic| observable: '+XX,(1, 1),+XY,+XY'
ic| observable: '+XX,(1, 1),+XZ,+X_'
ic| observable: '+XX,(1, 1),+XZ,+_Z'
ic| observable: '+XX,(1, 1),+XZ,+XZ'
ic| observable: '+XX,(1, 1),+YX,+Y_'
ic| observable: '+XX,(1, 1),+YX,+_X'
ic| observable: '+XX,(1, 1),+YX,+YX'
ic| observable: '+XX,(1, 1),+YY,+Y_'
ic| observable: '+XX,(1, 1),+YY,+_Y'
ic| observable: '+XX,(1, 1),+YY,+YY'
ic| observable: '+XX,(1, 1),+YZ,+Y_'
ic| observable: '+XX,(1, 1),+YZ,+_Z'
ic| observable: '+XX,(1, 1),+YZ,+YZ'
ic| observable: '+XX,(1, 1),+ZX,+Z_'
ic| observable: '+XX,(1, 1),+ZX,+_X'
ic| observable: '+XX,(1, 1),+ZX,+ZX'
ic| observable: '+XX,(1, 1),+ZY,+Z_'
ic| observable: '+XX,(1, 1),+ZY,+_Y'
ic| observable: '+XX,(1, 1),+ZY,+ZY'
ic| observable: '+XX,(1, 1),+ZZ,+Z_'
ic| observable: '+XX,(1, 1),+ZZ,+_Z'
ic| observable: '+XX,(1, 1),+ZZ,+ZZ'
i

In [8]:
jacobian_coef_dict["columns"]
jacobian_coef_dict["index"]

OrderedSet(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX', '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY', '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ', '+XX,(1, 1),+YX,+Y_', '+XX,(1, 1),+YX,+_X', '+XX,(1, 1),+YX,+YX', '+XX,(1, 1),+YY,+Y_', '+XX,(1, 1),+YY,+_Y', '+XX,(1, 1),+YY,+YY', '+XX,(1, 1),+YZ,+Y_', '+XX,(1, 1),+YZ,+_Z', '+XX,(1, 1),+YZ,+YZ', '+XX,(1, 1),+ZX,+Z_', '+XX,(1, 1),+ZX,+_X', '+XX,(1, 1),+ZX,+ZX', '+XX,(1, 1),+ZY,+Z_', '+XX,(1, 1),+ZY,+_Y', '+XX,(1, 1),+ZY,+ZY', '+XX,(1, 1),+ZZ,+Z_', '+XX,(1, 1),+ZZ,+_Z', '+XX,(1, 1),+ZZ,+ZZ', '+XY,(1, -1),+XX,+X_', '+XY,(1, -1),+XX,+_X', '+XY,(1, -1),+XX,+XX', '+XY,(1, -1),+XY,+X_', '+XY,(1, -1),+XY,+_Y', '+XY,(1, -1),+XY,+XY', '+XY,(1, -1),+XZ,+X_', '+XY,(1, -1),+XZ,+_Z', '+XY,(1, -1),+XZ,+XZ', '+XY,(1, -1),+YX,+Y_', '+XY,(1, -1),+YX,+_X', '+XY,(1, -1),+YX,+YX', '+XY,(1, -1),+YY,+Y_', '+XY,(1, -1),+YY,+_Y', '+XY,(1, -1),+YY,+YY', '+XY,(1, -1),+YZ,+Y_', '+XY,(1, -1),+YZ,+_Z', '+X

In [9]:
df.to_csv("ick.csv")

In [10]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [11]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, idle_string=global_idle_string
    )
idle_experiments

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

[Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1Gi:0:1[Gy:0Gy

In [12]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [ ]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    # ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [14]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

In [15]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [16]:
df.index

Index(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX',
       '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY',
       '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ',
       '+XX,(1, 1),+YX,+Y_',
       ...
       '+YX,(-1, -1),+YZ,+YZ', '+YX,(-1, -1),+ZX,+Z_', '+YX,(-1, -1),+ZX,+_X',
       '+YX,(-1, -1),+ZX,+ZX', '+YX,(-1, -1),+ZY,+Z_', '+YX,(-1, -1),+ZY,+_Y',
       '+YX,(-1, -1),+ZY,+ZY', '+YX,(-1, -1),+ZZ,+Z_', '+YX,(-1, -1),+ZZ,+_Z',
       '+YX,(-1, -1),+ZZ,+ZZ'],
      dtype='object', length=108)

In [17]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'XX', ' XX'): -5.1730544255886464e-17,
 ('XX', 'XY', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('XX', 'XZ', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XZ', ' _Z'): 0.0,
 ('XX', 'XZ', ' XZ'): 0.0,
 ('XX', 'YX', ' Y_'): 0.0,
 ('XX', 'YX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'YX', ' YX'): 0.0,
 ('XX', 'YY', ' Y_'): 0.0,
 ('XX', 'YY', ' _Y'): 0.0,
 ('XX', 'YY', ' YY'): 0.0,
 ('XX', 'YZ', ' Y_'): 0.0,
 ('XX', 'YZ', ' _Z'): 0.0,
 ('XX', 'YZ', ' YZ'): 0.0,
 ('XX', 'ZX', ' Z_'): 0.0,
 ('XX', 'ZX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'ZX', ' ZX'): 0.0,
 ('XX', 'ZY', ' Z_'): 0.0,
 ('XX', 'ZY', ' _Y'): 0.0,
 ('XX', 'ZY', ' ZY'): 0.0,
 ('XX', 'ZZ', ' Z_'): 0.0,
 ('XX', 'ZZ', ' _Z'): 0.0,
 ('XX', 'ZZ', ' ZZ'): 0.0,
 ('XY', 'XX', ' X_'): -5.1730544255886464e-17,
 ('XY', 'XX', ' _X'): 0.0,
 ('XY', 'XX', ' XX'): 0.0,
 ('XY', 'XY', ' X_'): 1.724077

In [18]:
j = df.to_numpy()

In [19]:
np.linalg.matrix_rank(j)

60

In [20]:
jinv = np.linalg.pinv(j)

In [21]:
intrins_errs = jinv @ obs_rats

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 972 is different from 108)

# Supposed intrinsic error rates (wrong)

In [ ]:
dict(zip(df.columns,intrins_errs))